In [1]:
# coding: utf-8
import os
import xlrd
import re
import csv
import fasttext
import traceback
import random
import utils
import json
import jieba
import pandas as pd
from utils import fixText

def getConfig():
    config = {}
    with open('./config.json', 'r') as f:
        s = f.read()
        config = json.loads(s)
    return config

In [2]:
with open('data/train.tsv', 'r') as f:
    content = f.readlines()
    random.shuffle(content)
    train = content[0:len(content) // 5 * 4]
    test = content[len(content) // 5 * 4:]
    
    with open('data/old_train_set.txt', 'w') as out:
        for c in train:
            res = re.search(r'__label__(.*)[^\r\n]', c)
            label = c[res.span()[0]:res.span()[1]]
            c = re.sub(r'__label__(.*)', '', c)
            c = ''.join(re.split('\t| |\r|\n', c))
            c = fixText(c)
            out.write(c + ' ' + label + '\n')
    with open('data/old_test_set.txt', 'w') as out:
        for c in test:
            res = re.search(r'__label__(.*)[^\r\n]', c)
            label = c[res.span()[0]:res.span()[1]]
            c = re.sub(r'__label__(.*)', '', c)
            c = ''.join(re.split('\t| |\r|\n', c))
            c = fixText(c)
            # c = ''.join(c.split(' '))
            out.write(c + ' ' + label + '\n')
        

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.728 seconds.
Prefix dict has been built successfully.


In [3]:
config = getConfig()
model = fasttext.train_supervised(
    input = './data/old_train_set.txt',
    lr = config['lr'],
    dim = config['hidden_dim'],
    epoch = config['epoch']
)

Read 4M words
Number of words:  73112
Number of labels: 43
Progress: 100.0% words/sec/thread:  800887 lr:  0.000000 avg.loss:  0.437031 ETA:   0h 0m 0s


In [11]:
total = 0
true_tag = 0
with open('./data/old_test_set.txt', 'r') as f:
    content = f.readlines()
    total = len(content)
    for c in content:
        res = re.search(r'__label__(.*)[^\r\n]', c)
        label = c[res.span()[0]:res.span()[1]]
        c = re.sub(r'__label__(.*)', '', c)
        predict = model.predict(c[:-1])[0][0]
        
        print(model.predict(c[:-1]))
        # print(predict, label)
        if(predict == label):
            true_tag += 1
        break
#print('tag level 1 accurate: {}% ({}/{})'.format(true_tag * 100 / total, true_tag, total))

(('__label__街道',), array([0.90377563]))


In [24]:
import re
def resetLabelLv2(label):
    if(re.search(r'.+街(道)*', label)):
        label = re.search(r'.+街(道)*', label).group()
        if "街道" not in label:
            label = label.replace("街", "街道")
        if(re.search(r'.+[区|市]', label)):
            label = re.sub(re.search(r'.+[区|市]', label).group(), '', label)
    return label

def resetLabelLv3(label):
    if(re.search(r'[\u4e00-\u9fa50-9]+街(道)*', label)):
        label = re.search(r'街(道)*.*', label).group()
        label = re.sub(r'[(|（][\u4e00-\u9fa50-9]*[)|）]', '', label)
    return label


In [27]:
addr = '广州市白云区区鹤洞街办事处假装有这个分队(测试一下括号内）测试一下括号外（测试一下多个括号）'
addr2 = resetLabelLv2(addr)
addr3 = resetLabelLv3(addr)
print(addr2)
print(addr3)


鹤洞街道
街办事处假装有这个分队测试一下括号外


In [1]:
import json
from utils import resetLabelLv2, resetLabelLv3
with open('./data/train_set.json', 'r') as f:
    s = f.read()
    data_set = json.loads(s)
label1 = {}
label2 = {}
label3 = {}

for id in list(data_set.keys()):
    if data_set[id]['tag_level_1'] in label1:
        label1[data_set[id]['tag_level_1']] += 1
    else:
        label1[data_set[id]['tag_level_1']] = 1

    tag = resetLabelLv2(data_set[id]['tag_level_2'])
    if tag in label2:
        label2[tag] += 1
    else:
        label2[tag] = 1

    tag = resetLabelLv3(data_set[id]['tag_level_3'])
    if tag in label3:
        label3[tag] += 1
    else:
        label3[tag] = 1

In [4]:
print(len(label1))
print(label1)

87
{'荔湾区政府': 144238, '广州地铁集团': 271, '番禺区政府': 244, '天河区政府': 536, '广州市自来水公司': 430, '市卫生健康委': 1424, '市交通运输局': 390, '白云区政府': 780, '广州商贸投资控股集团有限公司': 5, '海珠区政府': 638, '越秀区政府': 1160, '市公安局': 572, '花都区政府': 132, '珠江实业集团': 84, '广州市税务局': 230, '中国移动广州分公司': 10, '市住房城乡建设局': 243, '黄埔区政府': 137, '穗康码工作专班': 120, '市民政局': 37, '市人力资源社会保障局': 132, '增城区政府': 72, '岭南商旅集团': 3, '市燃气集团': 35, '市排水公司': 108, '市邮政管理局': 117, '广州市公共交通集团有限公司': 78, '南沙区政府': 136, '广州交投集团': 27, '市规划和自然资源局': 68, '市教育局': 112, '广州供电局有限公司': 240, '市来穗人员服务管理局': 41, '市医保局': 71, '市市场监管局': 14, '市国资委': 29, '中国联通广州分公司': 6, '从化区政府': 14, '市财政局': 2, '市水务局': 5, '市地方金融监管局': 45, '广州市消费者委员会': 4, '广州市烟草专卖局': 7, '中国电信广州分公司': 18, '市残联': 6, '市政务服务数据管理局': 8, '12345热线管理机构': 33, '农业银行广东省分行': 4, '市司法局': 4, '市城市管理综合执法局': 25, '市林业园林局': 14, '市总工会': 10, '越秀集团': 19, '广州市广播电视台': 2, '市发展改革委': 6, '广州轻工工贸集团': 17, '广州海事局': 2, '市体育局': 5, '市农业农村局': 1, '广州环保投资集团': 6, '市工业和信息化局': 1, '中国广电广州网络股份有限公司': 7, '市供销合作总社': 5, '市消防支队': 2, '工商银行广东省分行': 3, '市建筑集团': 9, '市港务局': 5, '市文化广电旅游局': 

In [4]:
import fasttext
model = fasttext.FastText.load_model('./model/model_for_label_2.model')
len(model.words)

146222